In [69]:
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
from IPython.display import display
import datetime
import re
import csv
from geopy.geocoders import Nominatim
from geopy.distance import geodesic # great_circle

# STATIC DATA

corps_rename_dict = {
    "Vanguard": "Santa Clara Vanguard",
    "Academy": "The Academy",
    "Cavaliers": "The Cavaliers",
    "Bushwackers": "Bushwackers Drum Corps",
    "Connecticut Hurricanes": "Hurricanes",
    "Battalion": "The Battalion"
}

corps_classes = ['DCI World Class', 'DCI Open Class','DCI All Age World Class','DCI All Age Open Class','DCI All Age Class A']

##################

dict_scores_table = pd.read_excel('new raw data/raw_data.xlsx', sheet_name=None, header=None)

for (k, table) in dict_scores_table.items():

    # general cleaning
    table.loc[1, 0] = "Corps"
    table.loc[1, 1] = "Performance Slot"
    date_and_place_and_comp = table.loc[0, 0]
    table.columns = table.loc[1]
    table.drop([0, 1, 2], inplace=True)

    # date and place
    comp = None
    if len(date_and_place_and_comp.split(sep='--')) > 1:
        (date_and_place, comp) = date_and_place_and_comp.split(sep='--')
    else:
        date_and_place = date_and_place_and_comp
    chunks = date_and_place.split(sep=' ')
    place = " ".join(chunks[1:])

    table.insert(0,column='Show Number',value=int(k))
    table.insert(1,column='Date',value=pd.to_datetime(chunks[0]))
    table.insert(2,column='Location',value=place)

    # removing categories
    table.drop([i for (i, row) in table.iterrows() if row['Corps'] in corps_classes], inplace=True)
    
    # type casting
    table['Performance Slot'] = table['Performance Slot'].astype(int)

scores_table = pd.DataFrame(columns=dict_scores_table['1'].columns)

for (k, v) in dict_scores_table.items():
    scores_table = scores_table.merge(v, how='outer')

def whitespace_remover(dataframe):
    for i in dataframe.columns:
        try:
            dataframe[i] = dataframe[i].map(str.strip)
        except:
            pass
# applying whitespace_remover function on dataframe
whitespace_remover(scores_table)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(scores_table)


1,Show Number,Date,Location,Corps,Performance Slot,General Effect 1,General Effect 2,General Effect Total,Visual Proficiency,Visual Analysis,Visual Color Guard,Visual Total,Music Brass,Music Analysis,Music Percussion,Music Total,Sub Total,Penalty,Total Score,General Effect 11,General Effect 1 2,General Effect,General Effect 2 1,General Effect 2 2,General Effect,Music Analysis 1,Music Analysis 2
0,1,2024-06-26,"Rockford, Michigan",Boston Crusaders,5,15.4,14.4,29.8,14.5,14.3,14.5,21.65,14.3,14.7,14.1,21.55,73,0,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2024-06-26,"Rockford, Michigan",Phantom Regiment,2,14.8,14.2,29,14,13.6,13.6,20.6,13.8,14.2,13.1,20.55,70.15,0,70.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2024-06-26,"Rockford, Michigan",Cavaliers,4,13.7,13.9,27.6,13.8,13.3,12.9,20,13.4,14.1,13.8,20.65,68.25,0,68.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2024-06-26,"Rockford, Michigan",Blue Stars,1,14.1,13.3,27.4,13.3,12.8,13.4,19.75,12.9,13.6,12.5,19.5,66.65,0,66.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2024-06-26,"Rockford, Michigan",Colts,3,13.8,13,26.8,13.4,12.6,12.8,19.4,12.7,13.3,12.4,19.2,65.4,0,65.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2,2024-06-28,"Muncie, Indiana",Carolina Crown,6,14.9,15.4,30.3,15.2,15,14.6,22.4,15.1,15.1,14.2,22.2,74.9,0,74.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2,2024-06-28,"Muncie, Indiana",Boston Crusaders,5,15,15,30,14.7,15.2,14.9,22.4,14.8,14.8,15,22.3,74.7,0,74.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2,2024-06-28,"Muncie, Indiana",Phantom Regiment,3,14.5,14.7,29.2,14.5,14.2,13.8,21.25,14.4,14.7,14.3,21.7,72.15,0,72.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2,2024-06-28,"Muncie, Indiana",Cavaliers,2,13.7,14.1,27.8,14,13.4,13.1,20.25,13.7,14.3,14.1,21.05,69.1,0,69.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2,2024-06-28,"Muncie, Indiana",Blue Stars,1,13.4,13.8,27.2,13.2,13.3,13.5,20,13.3,13.8,12.9,20,67.2,0,67.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
